<a href="https://colab.research.google.com/github/kriti-nigam/LSTM/blob/main/MultiClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re     
import nltk  
import string 
import warnings 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import emoji

In [5]:
!ls

aspell.txt  contractions.csv  sample_data  text_emotion.csv


In [6]:
!pip install tweet-preprocessor 2>/dev/null 1>/dev/null

In [7]:
data = pd.read_csv("text_emotion.csv")

In [8]:
data.head(20)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


In [9]:
#Pre Processing

In [10]:
#Misspelled words
misspell_data = pd.read_csv("aspell.txt",sep=":",names=["correction","misspell"])
misspell_data.misspell = misspell_data.misspell.str.strip()
misspell_data.misspell = misspell_data.misspell.str.split(" ")
misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
misspell_data.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

In [11]:
{v:miss_corr[v] for v in [list(miss_corr.keys())[k] for k in range(50)]}

{'Steffen': 'Stephen',
 'abilitey': 'ability',
 'abouy': 'about',
 'absorbtion': 'absorption',
 'accidently': 'accidentally',
 'accomodate': 'accommodate',
 'acommadate': 'accommodate',
 'acord': 'accord',
 'adultry': 'adultery',
 'aggresive': 'aggressive',
 'alchohol': 'alcohol',
 'alchoholic': 'alcoholic',
 'allieve': 'alive',
 'alright': 'all_right',
 'amature': 'amateur',
 'ambivilant': 'ambivalent',
 'amification': 'amplification',
 'amourfous': 'amorphous',
 'annoint': 'anoint',
 'annonsment': 'announcement',
 'annuncio': 'announce',
 'anonomy': 'anatomy',
 'anotomy': 'anatomy',
 'anynomous': 'anonymous',
 'appreceiated': 'appreciated',
 'appresteate': 'appreciate',
 'aquantance': 'acquaintance',
 'aratictature': 'architecture',
 'archeype': 'archetype',
 'aricticure': 'architecture',
 'artic': 'arctic',
 'asentote': 'asymptote',
 'ast': 'at',
 'asterick': 'asterisk',
 'asymetric': 'asymmetric',
 'atentively': 'attentively',
 'autoamlly': 'automatically',
 'bankrot': 'bankrupt',


In [12]:
def misspelled_correction(val):
    for x in val.split(): 
        if x in miss_corr.keys(): 
            val = val.replace(x, miss_corr[x]) 
    return val

data["clean_content"] = data.content.apply(lambda x : misspelled_correction(x))

In [13]:
#Removing COntractions

In [14]:
contractions = pd.read_csv("contractions.csv")
cont_dic = dict(zip(contractions.Contraction, contractions.Meaning))

In [15]:
def cont_to_meaning(val): 
  
    for x in val.split(): 
        if x in cont_dic.keys(): 
            val = val.replace(x, cont_dic[x]) 
    return val

In [16]:
data.clean_content = data.clean_content.apply(lambda x : cont_to_meaning(x))

In [17]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,Re-pinging @ghostridah14: why did not you go t...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...","I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,@kelcouch I am sorry at least it is Friday?


In [18]:
#Removing URLs and Usernames

In [19]:

import re
import preprocessor as p   



In [20]:
p.set_options(p.OPT.MENTION, p.OPT.URL)
p.clean("hello guys @alx #sport🔥 1245 https://github.com/s/preprocessor")

'hello guys #sport🔥 1245'

In [21]:
data["clean_content"]=data.content.apply(lambda x : p.clean(x))

In [22]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier and...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin on...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,Re-pinging : why didn't you go to prom? BC my ...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...","I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,Hmmm. is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,I'm sorry at least it's Friday?


In [23]:
#Removing Punctuations and Emojis

In [24]:
def punctuation(val): 
  
    punctuations = '''()-[]{};:'"\,<>./@#$%^&_~'''
  
    for x in val.lower(): 
        if x in punctuations: 
            val = val.replace(x, " ") 
    return val

In [25]:
punctuation("test @ #ldfldlf??? !! ")

'test    ldfldlf??? !! '

In [28]:
import emoji

ModuleNotFoundError: ignored

In [1]:
!setup.py install

/bin/bash: setup.py: command not found


In [2]:
!pip install emoji


     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=ee29c7531ed278d8566df8ca9a6f366dd086d2186ccf0944ea2b8ba04b8c4595
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [3]:
import emoji

In [26]:
data.clean_content = data.clean_content.apply(lambda x : ' '.join(punctuation(emoji.demojize(x)).split()))

In [28]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier and...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh waitin on y...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony gloomy friday
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,Re pinging why didn t you go to prom? BC my bf...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...",I should be sleep but im not! thinking about a...
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,Hmmm is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,Charlene my love I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,I m sorry at least it s Friday?


In [29]:
def clean_text(val):
    val = misspelled_correction(val)
    val = cont_to_meaning(val)
    val = p.clean(val)
    val = ' '.join(punctuation(emoji.demojize(val)).split())
    
    return val

In [30]:
clean_text("isn't 💡 adultry @ttt good bad ... ! ? ")

'is not light bulb adultery good bad ! ?'

In [31]:
#Removing Empty Comments

In [32]:
data = data[data.clean_content != ""]

In [33]:
data.head(20)

,tweet_id,sentiment,author,content,clean_content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier and...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh waitin on y...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony gloomy friday
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...,Re pinging why didn t you go to prom? BC my bf...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ...",I should be sleep but im not! thinking about a...
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down,Hmmm is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you,Charlene my love I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?,I m sorry at least it s Friday?


In [34]:
data.sentiment.value_counts()

neutral       8579
worry         8454
happiness     5208
sadness       5162
love          3841
surprise      2187
fun           1776
relief        1526
hate          1323
empty          815
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64